<h1 style="text-align: center; font-weight: bold;">TP2: Basic Text Processing</h1>
<h3 style="font-weight: bold;">Exercise</h3>
<p>Using the NLTK (Natural Language Toolkit) package, we want to enrich the text (text.html) with a set of annotations such as the number of occurrences in the text, the lemma, the root, the grammatical category, etc. The result should be stored in a CSV file. Each line of the result file should contain the word and all the labels (see annotated_text.csv).</p>
<p>Tasks to be completed:</p>
<ul>
  <li>Using regular expressions, filter the "text.html" file to remove tags and non-important information. Keep only the text.</li>
  <li>Segment the resulting text into sentences.</li>
  <li>Segment each sentence into words.</li>
  <li>For each word, add the necessary labels according to the result file (see the "annotated_text.csv" file).</li>
  <li>Repeat the same process, this time using the spaCy package.</li>
</ul>

### **Import Libraries**

In [431]:
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.chunk import ne_chunk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import spacy
from IPython.display import display

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to

True

### **Preprocess Text**

In [432]:
def read_file(filename):
    """
    Reads and processes a text file.

    Args:
        filename (str): The name of the file to be read.

    Returns:
        str : File content
    """
    # Read the file and return a list of lines
    with open(filename, 'r') as f:
        return f.read()

In [433]:
text = read_file('./text.html')
text

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8"/>\n\t\t<h1>\n\t\t\t<b> The Providence Journal</b>\n\t\t</h1>\n\t</head>\n\t<body>\n\t\t<text>\n\t\t\t<p>East Providence should organize its civil defense setup and begin by appointing a full-time director, Raymond H. Hawksley, the present city CD head, believes. Mr. Hawksley said yesterday he would be willing to go before the city council `` or anyone else locally \'\' to outline his proposal at the earliest possible time. East Providence now has no civil defense program. Mr. Hawksley, the state\'s general treasurer, has been a part-time CD director in the city for the last nine years. He is not interested in being named a full-time director. </p>\n\t\t\t<p>Noting that President Kennedy has handed the Defense Department the major responsibility for the nation\'s civil defense program, Mr. Hawksley said the federal government would pay half the salary of a full-time local director. He expressed the opinion the city could hire

In [434]:
def remove_html_tags(text):
    """Remove HTML tags from a given text.

    Args:
        text (str): The input text containing HTML tags.

    Returns:
        str: The text with HTML tags removed.
    """
    if text is None:
        return ""

    if not isinstance(text, str):
        text = str(text)
        
    # Define a regular expression to match HTML tags
    html_tags_pattern = re.compile(r'<.*?>')
    
    # Use the sub() function to replace HTML tags with an empty string
    text_without_html = re.sub(html_tags_pattern, '', text)
    
    return text_without_html


In [435]:
text_without_tags = remove_html_tags(text=text)
text_without_tags

"\n\n\t\n\t\t\n\t\t\n\t\t\t The Providence Journal\n\t\t\n\t\n\t\n\t\t\n\t\t\tEast Providence should organize its civil defense setup and begin by appointing a full-time director, Raymond H. Hawksley, the present city CD head, believes. Mr. Hawksley said yesterday he would be willing to go before the city council `` or anyone else locally '' to outline his proposal at the earliest possible time. East Providence now has no civil defense program. Mr. Hawksley, the state's general treasurer, has been a part-time CD director in the city for the last nine years. He is not interested in being named a full-time director. \n\t\t\tNoting that President Kennedy has handed the Defense Department the major responsibility for the nation's civil defense program, Mr. Hawksley said the federal government would pay half the salary of a full-time local director. He expressed the opinion the city could hire a CD director for about $3,500 a year and would only have to put up half that amount on a matching

In [436]:
def decompose_text_into_lines(text):
    """
    Decompose a text into lines, removing empty lines and leading/trailing whitespace.

    Parameters:
    text (str): The input text to be decomposed into lines.

    Returns:
    list of str: A list of lines from the input text.
    """
    
    # Split the text into lines using newline as the delimiter
    sentences = text.split('\n')
    
    # Remove leading/trailing whitespace and empty lines
    sentences = [s.replace('\t', '').strip() for s in sentences if s.strip() != '']
    
    return sentences

In [437]:
sentences = decompose_text_into_lines(text_without_tags)
sentences

['The Providence Journal',
 "East Providence should organize its civil defense setup and begin by appointing a full-time director, Raymond H. Hawksley, the present city CD head, believes. Mr. Hawksley said yesterday he would be willing to go before the city council `` or anyone else locally '' to outline his proposal at the earliest possible time. East Providence now has no civil defense program. Mr. Hawksley, the state's general treasurer, has been a part-time CD director in the city for the last nine years. He is not interested in being named a full-time director.",
 "Noting that President Kennedy has handed the Defense Department the major responsibility for the nation's civil defense program, Mr. Hawksley said the federal government would pay half the salary of a full-time local director. He expressed the opinion the city could hire a CD director for about $3,500 a year and would only have to put up half that amount on a matching fund basis to defray the salary costs.",
 'Mr. Hawks

### **NLTK vs SpaCy**

#### **1. NLTK**

In [438]:
def get_wordnet_pos(tag):
    tag = tag[0].upper()
    tag_dict = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    return tag_dict.get(tag, wordnet.NOUN)

In [439]:
def named_entities_nltk(tokens):
    """
    Extract named entities from a list of tokens using NLTK.

    Parameters:
    tokens (list): A list of tokens representing a sentence.

    Returns:
    list: A list of strings representing named entities with their labels.
    """

    # Initialize a dictionary to store named entities
    namentities = dict()
    
    # Perform named entity chunking
    ne = ne_chunk(nltk.pos_tag(tokens))
    i = 0
    # Iterate over entities in the named entity chunk
    for entity in ne:
        # Check if the entity has a label (indicating it's a named entity)
        if hasattr(entity, 'label'):
            # Get the leaves (individual words) of the entity
            leaves = entity.leaves()
            # Iterate over leaves and store them in the named_entities dictionary
            for leaf in leaves:
                # Combine the word and label information
                if leaf[0] not in namentities.keys():
                    i += 1
                    namentities[leaf[0]] = ''.join([leaf[-1]+str(i), ' - ', entity.label()])
        else:
            # If the entity does not have a label, store it in the dictionary with a placeholder label "-"
            if entity[0] not in namentities.keys():
                namentities[entity[0]] = '-'
    # Return a list of named entities with their labels
    return namentities.values()

In [440]:
def annotate_text_with_nltk(sentences):
    """
    Annotate text using NLTK for information like word, occurrences, lemma, stem, POS tag, and stopword status.

    Parameters:
    sentences (list): A list of sentences or text to be annotated.

    Returns:
    pandas.DataFrame: A DataFrame containing annotations, and also saves the annotations to an Excel file.
    """

    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    stop_words = set(stopwords.words('english'))
    results_df = []

    for sentence in sentences:
        # Tokenize the sentence
        tokens = word_tokenize(sentence)
        
        # Initialize dictionaries to store annotations
        annotations = {'word': [], 'occurrences': [], 'lemma': [], 'stem': [], 
                    'pos_tag': [], 'named_entity': [], 'is_stopword': []}
        word_occurrence = {}

        for token in tokens:
            # Count word occurrences
            word_occurrence[token] = word_occurrence.get(token, 1) + 1

            if token == '' or token in annotations['word']:
                continue

            annotations['word'].append(token)
            annotations['is_stopword'].append(token in stop_words)
            pos_tag = nltk.pos_tag([token])[0][1]
            annotations['pos_tag'].append(pos_tag)
            annotations['lemma'].append(lemmatizer.lemmatize(token, get_wordnet_pos(pos_tag)))
            annotations['stem'].append(stemmer.stem(token))
        
        
        annotations['named_entity'] = named_entities_nltk(tokens=tokens)

        # Calculate word occurrences
        for word in annotations['word']:
            annotations['occurrences'].append(word_occurrence[word])

        # Create a DataFrame from the annotations
        annotations_df = pd.DataFrame(annotations)
        display(annotations_df)
        results_df.append(annotations_df)

    annotated_text = pd.concat(results_df, axis=0, ignore_index=True)
        
    # Save the annotations to an Excel file
    annotated_text.to_excel('text_annotations_using_nltk.xlsx')

In [441]:
annotate_text_with_nltk(sentences)

,word,occurrences,lemma,stem,pos_tag,named_entity,is_stopword
0,The,2,The,the,DT,-,False
1,Providence,2,Providence,provid,NN,NNP1 - ORGANIZATION,False
2,Journal,2,Journal,journal,NNP,NNP2 - ORGANIZATION,False


,word,occurrences,lemma,stem,pos_tag,named_entity,is_stopword
0,East,3,East,east,NN,JJ1 - GPE,False
1,Providence,3,Providence,provid,NN,NNP2 - ORGANIZATION,False
2,should,2,should,should,MD,-,True
3,organize,2,organize,organ,VB,-,False
4,its,2,it,it,PRP$,-,True
...,...,...,...,...,...,...,...
65,is,2,be,is,VBZ,-,True
66,not,2,not,not,RB,-,True
67,interested,2,interested,interest,JJ,-,False
68,being,2,be,be,VBG,-,True


,word,occurrences,lemma,stem,pos_tag,named_entity,is_stopword
0,Noting,2,Noting,note,VBG,-,False
1,that,3,that,that,IN,-,True
2,President,2,President,presid,NNP,-,False
3,Kennedy,2,Kennedy,kennedi,NN,NNP1 - PERSON,False
4,has,2,have,has,VBZ,-,True
5,handed,2,hand,hand,VBN,-,False
6,the,9,the,the,DT,-,True
7,Defense,2,Defense,defens,NN,NNP2 - ORGANIZATION,False
8,Department,2,Department,depart,NNP,-,False
9,major,2,major,major,JJ,-,False


,word,occurrences,lemma,stem,pos_tag,named_entity,is_stopword
0,Mr.,3,Mr.,mr.,NNP,NNP1 - PERSON,False
1,Hawksley,3,Hawksley,hawksley,NNP,NNP2 - PERSON,False
2,said,6,say,said,VBD,-,False
3,he,5,he,he,PRP,-,True
4,believed,2,believe,believ,VBN,-,False
...,...,...,...,...,...,...,...
79,radios,2,radio,radio,NNS,-,False
80,sirens,2,siren,siren,NNS,-,False
81,and,2,and,and,CC,-,True
82,rescue,2,rescue,rescu,NN,-,False


,word,occurrences,lemma,stem,pos_tag,named_entity,is_stopword
0,Mr.,3,Mr.,mr.,NNP,NNP1 - PERSON,False
1,Hawksley,3,Hawksley,hawksley,NNP,NNP2 - ORGANIZATION,False
2,believes,2,believe,believ,VBZ,-,False
3,that,3,that,that,IN,-,True
4,East,2,East,east,NN,NNP3 - PERSON,False
...,...,...,...,...,...,...,...
61,go,2,go,go,VB,-,False
62,event,2,event,event,NN,-,False
63,an,2,an,an,DT,-,True
64,enemy,2,enemy,enemi,NN,-,False


#### **2. SpaCy**

In [442]:
def annotate_text_with_spacy(sentences):
    """
    Annotate text using spaCy for information like word, occurrences, lemma, POS tag, and stopword status.

    Parameters:
    sentences (list): A list of sentences or text to be annotated.

    Returns:
    pandas.DataFrame: A DataFrame containing annotations, and also saves the annotations to an Excel file.
    """
    
    # Load the English language model
    nlp = spacy.load("en_core_web_sm")
    results_df = []    

    for sentence in sentences:
        # Initialize dictionaries to store annotations
        annotations = {'word': [], 'occurrences': [], 'lemma': [], 'pos_tag': [], 
                    'named_entity': [], 'is_stopword': []}
        word_occurrence = {}
        # Process the text using spaCy
        tokens = nlp(sentence)
        i = 0
        
        for token in tokens:

            word_occurrence[str(token)] = word_occurrence.get(str(token), 1) + 1

            if token.text == '' or token.text in annotations['word']:
                continue

            annotations['word'].append(str(token))
            annotations['is_stopword'].append(token.is_stop)
            annotations['pos_tag'].append(token.pos_)
            annotations['lemma'].append(token.lemma_)
            # Add named entity information
            if token.ent_type_:
                i += 1
                annotations['named_entity'].append(f'NNP{i} - ' + token.ent_type_)
            else:
                annotations['named_entity'].append('-')

        # Calculate word occurrences
        for word in annotations['word']:
            annotations['occurrences'].append(word_occurrence[word])

        # Create a DataFrame from the annotations
        annotations_df = pd.DataFrame(annotations)
        display(annotations_df)
        results_df.append(annotations_df)

    annotated_text = pd.concat(results_df, axis=0, ignore_index=True)

    # Save the annotations to an Excel file
    annotated_text.to_excel('text_annotations_using_spacy.xlsx')


In [443]:
annotate_text_with_spacy(sentences)

,word,occurrences,lemma,pos_tag,named_entity,is_stopword
0,The,2,the,DET,NNP1 - ORG,True
1,Providence,2,Providence,PROPN,NNP2 - ORG,False
2,Journal,2,Journal,PROPN,NNP3 - ORG,False


,word,occurrences,lemma,pos_tag,named_entity,is_stopword
0,East,3,East,PROPN,NNP1 - GPE,False
1,Providence,3,Providence,PROPN,NNP2 - GPE,False
2,should,2,should,AUX,-,True
3,organize,2,organize,VERB,-,False
4,its,2,its,PRON,-,True
...,...,...,...,...,...,...
67,is,2,be,AUX,-,True
68,not,2,not,PART,-,True
69,interested,2,interested,ADJ,-,False
70,being,2,be,AUX,-,True


,word,occurrences,lemma,pos_tag,named_entity,is_stopword
0,Noting,2,note,VERB,-,False
1,that,3,that,SCONJ,-,True
2,President,2,President,PROPN,-,False
3,Kennedy,2,Kennedy,PROPN,NNP1 - PERSON,False
4,has,2,have,AUX,-,True
5,handed,2,hand,VERB,-,False
6,the,9,the,DET,NNP2 - ORG,True
7,Defense,2,Defense,PROPN,NNP3 - ORG,False
8,Department,2,Department,PROPN,NNP4 - ORG,False
9,major,2,major,ADJ,-,False


,word,occurrences,lemma,pos_tag,named_entity,is_stopword
0,Mr.,3,Mr.,PROPN,-,False
1,Hawksley,3,Hawksley,PROPN,NNP1 - PERSON,False
2,said,6,say,VERB,-,False
3,he,5,he,PRON,-,True
4,believed,2,believe,VERB,-,False
...,...,...,...,...,...,...
81,radios,2,radio,NOUN,-,False
82,sirens,2,siren,NOUN,-,False
83,and,2,and,CCONJ,-,True
84,rescue,2,rescue,NOUN,-,False


,word,occurrences,lemma,pos_tag,named_entity,is_stopword
0,Mr.,3,Mr.,PROPN,-,False
1,Hawksley,3,Hawksley,PROPN,NNP1 - PERSON,False
2,believes,2,believe,VERB,-,False
3,that,3,that,SCONJ,-,True
4,East,2,East,PROPN,NNP2 - GPE,False
...,...,...,...,...,...,...
61,go,2,go,VERB,-,True
62,event,2,event,NOUN,-,False
63,an,2,an,DET,-,True
64,enemy,2,enemy,NOUN,-,False


#### **3. Difference between NLTK and SpaCy**

<table border="1">
  <tr>
    <th>Feature</th>
    <th>NLTK</th>
    <th>spaCy</th>
  </tr>
  <tr>
    <td>Tokenization</td>
    <td>Word and sentence tokenization functions are available.</td>
    <td>Advanced tokenization with word and sentence boundary detection.</td>
  </tr>
  <tr>
    <td>Lemmatization</td>
    <td>Basic lemmatization is available but may require additional custom rules.</td>
    <td>Built-in lemmatization with WordNet integration and support for various languages.</td>
  </tr>
  <tr>
    <td>Stemming</td>
    <td>Provides stemming algorithms for English and other languages.</td>
    <td>Lemmatization is provided, but stemming is not available in spaCy.</td>
  </tr>
  <tr>
    <td>Part-of-Speech (POS) Tagging</td>
    <td>Provides POS tagging but may require additional data for more languages.</td>
    <td>Built-in POS tagging for multiple languages with pre-trained models.</td>
  </tr>
  <tr>
    <td>Stopword Removal</td>
    <td>Stopword lists are available, and removal can be customized.</td>
    <td>Built-in stopword removal with support for multiple languages.</td>
  </tr>
  <tr>
    <td>Named Entity Recognition (NER)</td>
    <td>NER functionality is available but may require additional training for specific domains.</td>
    <td>Built-in NER with support for various entity types and languages.</td>
  </tr>
  <tr>
    <td>Dependency Parsing</td>
    <td>Basic dependency parsing with customizable grammar rules.</td>
    <td>Advanced dependency parsing with pre-trained models and efficient parsing algorithms.</td>
  </tr>
  <tr>
    <td>Customization</td>
    <td>NLTK allows you to define custom rules for various NLP tasks.</td>
    <td>spaCy provides more advanced customization options, including training new models.</td>
  </tr>
  <tr>
    <td>Community and Documentation</td>
    <td>Has an active community and extensive documentation.</td>
    <td>Has a growing community and comprehensive documentation.</td>
  </tr>
</table>